In [6]:
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)


class CalculatorToolArgsSchema(BaseModel):
  a: float = Field(description="The First number")
  b: float = Field(description="The Second number")

class CalculatorTool(BaseTool):
  name = "CalculatorTool"
  description = """
    Use this to perform sums of two numbers.
    The first and second arguments should be numbers.
    Only receives two arguments.
    """
  args_schema: Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema

  def _run(self,a, b):
    return a + b

agent = initialize_agent(
  llm=llm, 
  verbose=True,
  agent=AgentType.OPENAI_FUNCTIONS,
  handle_parsing_errors=True,
  tools=[CalculatorTool()]
)

prompt = "Cost of $123.42 + $12932.12 + $121.3 + $120.59 + $1940.29 + $59.11"

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `CalculatorTool` with `{'a': 123.42, 'b': 12932.12}`


13055.54
Invoking: `CalculatorTool` with `{'a': 13055.54, 'b': 121.3}`


13176.84
Invoking: `CalculatorTool` with `{'a': 13176.84, 'b': 120.59}`


13297.43
Invoking: `CalculatorTool` with `{'a': 13297.43, 'b': 1940.29}`


15237.720000000001
Invoking: `CalculatorTool` with `{'a': 15237.72, 'b': 59.11}`


15296.83The total cost of $123.42 + $12932.12 + $121.3 + $120.59 + $1940.29 + $59.11 is $15296.83.

> Finished chain.


{'input': 'Cost of $123.42 + $12932.12 + $121.3 + $120.59 + $1940.29 + $59.11',
 'output': 'The total cost of $123.42 + $12932.12 + $121.3 + $120.59 + $1940.29 + $59.11 is $15296.83.'}